<a href="https://colab.research.google.com/github/hieunguyen-cyber/AI_Project_CBH_GHHKM/blob/main/Predicting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sử dụng mô hình YOLOv10 đã được huấn luyện để nhận diện biên báo giao thông**


Ở đây, chúng ta sẽ sử dụng file trọng số **best.pt** thu được trong giai đoạn huấn luyện để áp dụng vào mô hình YOLOv10 nhằm giải quyết bài toán nhận diện biển báo giao thông.

**Chú ý**: Kết nối với GPU trước khi sử dụng

Nhóm thực hiện: CBH_GHHKM

---

###**Bước 1**: Tải file trọng số **best.pt** lên Google Colab

In [ ]:
!git clone https://github.com/hieunguyen-cyber/Weight_AIBK_2024.git
!mv /content/Weight_AIBK_2024/best.pt /content

Có thể sử dụng trọng số ở các phiên bản huấn luyện trước qua đường link sau: [best_version](https://drive.google.com/drive/folders/1Skn6lKOL8cTzaCyB9m3RbG_lRAWvNBZL?usp=sharing)

###**Bước 2**: Tải dữ liệu muốn dự đoán

Để các ảnh cần dự đoán trong 1 folder tên **test_img/test_vid** trong Google Drive

* Có thể tham khảo bộ ảnh sau: [test_img](https://l.messenger.com/l.php?u=https%3A%2F%2Fdrive.google.com%2Fdrive%2Ffolders%2F1totO1tw2wZ_Fv2RSG72_4tuUZC66LdpY%3Fusp%3Ddrive_link&h=AT1vv7_RcZ3qzSh9Wt_YqVTeYw9jVWh1ibVWN_Egp3bO1hhzk_8POVws0soYLXyYsVMgxOIMmyJxYB26wukr3E_dMXS6biCLG34T6ZjkYm-Fmt2djZGnzr4eHfexZl-rgLFb1g)

* Có thể tham khảo bộ video sai: [test_vid](https://drive.google.com/drive/folders/1AnmtegDkE8LV1Wed94Tkc7MVO1EroX2Q?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###**Bước 3**: Cài đặt YOLOv10

In [ ]:
!git clone https://github.com/THU-MIG/yolov10

###**Bước 4**: Tiến hành dự đoán

 Nếu muốn sử dụng các phiên bản trọng số trước, thay **MODEL_PATH** thành
 **/content/drive/MyDrive/best_version/best_ver_x** với x là phiên bàn muốn chạy.

* Đối với ảnh

In [ ]:
%cd /content/yolov10

from ultralytics import YOLOv10
import os

MODEL_PATH = '/content/best.pt'
model = YOLOv10(MODEL_PATH)

input_folder = '/content/drive/MyDrive/test_img'
output_folder = '/content/output_images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        input_path = os.path.join(input_folder, filename)

        results = model.predict(input_path)

        output_path = os.path.join(output_folder, filename)
        results[0].save(output_path)

        print(f'Processed and saved: {output_path}')


Ảnh đã dự đoán được lưu tại folder **output_images**

* Đối với video

In [ ]:
from ultralytics import YOLOv10
import os
import cv2
import numpy as np

# Paths to model and directories
MODEL_PATH = '/content/best.pt'
input_folder = '/content/drive/MyDrive/test_vid'
output_folder = '/content/output_videos'

# Load the YOLO model
model = YOLOv10(MODEL_PATH)

# Create the output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each video file in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Open the video file
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error opening video file: {input_path}")
            continue

        # Get video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi files
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB format for YOLO
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Make predictions
            results = model.predict(frame_rgb)

            # Draw results on the frame
            annotated_frame = results[0].plot()  # Plotting adds annotations to the frame

            # Convert back to BGR format for saving
            annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

            # Write the frame to the output video
            out.write(annotated_frame_bgr)

        # Release resources
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        print(f'Processed and saved: {output_path}')


Ảnh đã dự đoán được lưu tại folder **output_videos**